In [3]:
import glob
import re
import os
import string
import time
import math
import sys
import json
from array import array
from _collections import defaultdict
import operator
import pickle
from sys import getsizeof
from operator import itemgetter
from audioop import reverse
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import PorterStemmer
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

#Create a dictionary in the form self.entry = {'term':t,'docFreq':df,'termFreq':tf,'postingList':pl}
class DictEntry:
    def __init__(self,t,df,tf,pl):
        self.term = t
        self.docFreq = df
        self.totTermFreq = tf
        self.postingList = pl

        
class PostingEntry:
    def __init__(self,did,freq,maxtf,docl):
        self.docId = did
        self.termFreq = freq
        self.maxTermFreq = maxtf
        self.docLen = docl 
    def __iter__(self):
        return self.__dict__.iteritems()

#Function to get unary code
def getUnaryValue(leng):
    unaryValue = ""
    for i in range(0,leng):
        unaryValue += str(1)
    return unaryValue + str(0)

#Function to get Delta code
def getDeltaCode(num):
    binaryRep = str(bin(num))[2:]
    gammaCode = getGammaCode(len(binaryRep))
    offset = binaryRep[1:]
    deltaCode = gammaCode + offset
    return deltaCode

#Function to get Gamma code
def getGammaCode(num):
    binaryRep = str(bin(num))[2:]
    offset = binaryRep[1:]
    unaryValue = getUnaryValue(len(offset))
    gammaCode = unaryValue + offset
    return gammaCode#byteGamma

#Function to get Maximum Term Frequency of Document code
def getMaxTermFreqDocLen(dictf):
    docLeng = 0
    maxTermFreq = 0
    for items in dictf:
        termFreq = dictf[items]
        docLeng += termFreq
        if not items in stopWords:
            if termFreq > maxTermFreq:
                maxTermFreq = termFreq
    return maxTermFreq,docLeng 
                
def insertDict(docId, token, termFreq, dictionary, maxTermFreq, docLeng):
    entry = dictionary.get(token)
    if entry is None:
        postingList = []
        entry = DictEntry(token,0,0,postingList)
        dictionary[token] = entry       

    entry.docFreq +=1
    postEntry = PostingEntry(docId,termFreq,maxTermFreq,docLeng)
    entry.postingList.append(postEntry)
    entry.totTermFreq+= termFreq

#Tokenization of text
def Tokenize(files):
    textFile = open(files,"r")
    lText = textFile.read().lower()
    plainWord = re.sub('<[^>]*>','', lText)
    text = re.sub(r'\.(?![a-zA-Z]{3})', '', plainWord)
    text = text.replace("\'s","")
    #text = text.replace("\'[a-z]+",' ')
    words =re.sub('[^a-zA-Z]+', ' ', text).split()
    # re.split(r'[-=\.,?!:$;_()\[\]\`\'*"/\t\n\r\d+ \x0b\x0c]+', text)
    ##re.sub(r"\p{P}+", "", text.lower()).split()#
    textFile.close()
    return [word.strip() for word in words if word.strip() != '']
        
#Lemmatiztion of text   
def Lemmatiztion():        
    lmtzr = WordNetLemmatizer()
    for docId ,dfile in enumerate(filesList,1): 
        dictWord = {}        
        listWord = Tokenize(dfile)
        for word in listWord:
                tWord = lmtzr.lemmatize(word)
                #print tWord
                dictWord[tWord] = dictWord.get(tWord,0)+ 1
        
        mxtf, dolen = getMaxTermFreqDocLen(dictWord)
        for items in dictWord:
            termFreq = dictWord[items]
            if not items in stopWords :    
                insertDict(docId,items,termFreq,dictionary_uncomp_v1,mxtf,dolen)

        del dictWord

#Perform stemming operation on text 
def Stemming():
    stmr = PorterStemmer()
    for docId ,dfile in enumerate(filesList,1):
        listWord = Tokenize(dfile)    
        stemWord = {}        
        for w in listWord:
            tempStem = stmr.stem(w) 
            stemWord[tempStem]= stemWord.get(tempStem,0)+1
        
        mxtf, dolen = getMaxTermFreqDocLen(stemWord)
        
        for items in stemWord:
            termFreq = stemWord[items]    
            if not items in stopWords :    
                insertDict(docId,items,termFreq,dictionary_uncomp_v2,mxtf,dolen)
        
        del stemWord

def getDocLargestMaxTF(dictionary_uncomp):
    maz=0
    for term in dictionary_uncomp.keys():
        entry = dictionary_uncomp.get(term)
        for pEntry in entry.postingList:
            if pEntry.maxTermFreq > maz:
                maz = pEntry.maxTermFreq

    for term in dictionary_uncomp.keys():
        entry = dictionary_uncomp.get(term)
        for pEntry in entry.postingList:
            if pEntry.maxTermFreq == maz:            
                return pEntry.docId,pEntry.maxTermFreq
            
def getDocLargestDocLen(dictionary_uncomp):
    doc=0
    for term in dictionary_uncomp.keys():
        entry = dictionary_uncomp.get(term)
        for pEntry in entry.postingList:
            if pEntry.docLen > doc:
                doc = pEntry.docLen

    for term in dictionary_uncomp.keys():
        entry = dictionary_uncomp.get(term)
        for pEntry in entry.postingList:
            if pEntry.docLen == doc:            
                return pEntry.docId,pEntry.docLen
            
#Blocked compression using gamma coding
def blockedCompression_gamma():#getGammaCode
    k=8
    tempK=0
    dictString = ""
    termFreqBlock = {}
    docFreqBlock = {}
    gammaEncodingList = []
    tempIndexList = []
    for f,term in enumerate(dictionary_uncomp_v1.keys()):
        if tempK < k:
            dictString += str(len(term)) + term
            entry = dictionary_uncomp_v1.get(term)
            prevId = 0
            pEntry = PostingEntry(0,0,0,0)
            for pEntry in entry.postingList:
                docId = getGammaCode(pEntry.docId - prevId)
                gammaEncodingList.append(docId)
                prevId = pEntry.docId
            termFreqBlock[tempK] = getGammaCode(entry.totTermFreq)
            docFreqBlock[tempK] = getGammaCode(entry.docFreq)
            tempK += 1
             
        if tempK == k or f == len(dictionary_uncomp_v1)-1:
            tempK=0
            tempIndexList.append(gammaEncodingList)
            tempIndexList.append(termFreqBlock)
            tempIndexList.append(docFreqBlock)
            compressedIndexV1[dictString] = tempIndexList 
            dictString = ""
            tempIndexList = []
            gammaEncodingList = []
            termFreqBlock = {}
            docFreqBlock = {}
                                
    return compressedIndexV1

#Blocked compression using delta coding
def blockedCompression_delta(): #getDeltaCode
    k=8
    tempK=0
    dictString = ""
    termFreqBlock = {}
    docFreqBlock = {}
    gammaEncodingList = []
    tempIndexList = []
    for f,term in enumerate(dictionary_uncomp_v1.keys()):
        if tempK < k:
            dictString += str(len(term)) + term
            entry = dictionary_uncomp_v1.get(term)
            prevId = 0
            pEntry = PostingEntry(0,0,0,0)
            for pEntry in entry.postingList:
                docId = getDeltaCode(pEntry.docId - prevId)
                gammaEncodingList.append(docId)
                prevId = pEntry.docId
            termFreqBlock[tempK] = getDeltaCode(entry.totTermFreq)
            docFreqBlock[tempK] = getDeltaCode(entry.docFreq)
            tempK += 1
             
        if tempK == k or f == len(dictionary_uncomp_v1)-1:
            tempK=0
            tempIndexList.append(gammaEncodingList)
            tempIndexList.append(termFreqBlock)
            tempIndexList.append(docFreqBlock)
            compressedIndexV1[dictString] = tempIndexList 
            dictString = ""
            tempIndexList = []
            gammaEncodingList = []
            termFreqBlock = {}
            docFreqBlock = {}
                                
    return compressedIndexV1

#Blocked compression using unary coding
def blockedCompression_unary(): #getUnaryValue
    k=8
    tempK=0
    dictString = ""
    termFreqBlock = {}
    docFreqBlock = {}
    gammaEncodingList = []
    tempIndexList = []
    for f,term in enumerate(dictionary_uncomp_v1.keys()):
        if tempK < k:
            dictString += str(len(term)) + term
            entry = dictionary_uncomp_v1.get(term)
            prevId = 0
            pEntry = PostingEntry(0,0,0,0)
            for pEntry in entry.postingList:
                docId = getUnaryValue(pEntry.docId - prevId)
                gammaEncodingList.append(docId)
                prevId = pEntry.docId
            termFreqBlock[tempK] = getUnaryValue(entry.totTermFreq)
            docFreqBlock[tempK] = getUnaryValue(entry.docFreq)
            tempK += 1
             
        if tempK == k or f == len(dictionary_uncomp_v1)-1:
            tempK=0
            tempIndexList.append(gammaEncodingList)
            tempIndexList.append(termFreqBlock)
            tempIndexList.append(docFreqBlock)
            compressedIndexV1[dictString] = tempIndexList 
            dictString = ""
            tempIndexList = []
            gammaEncodingList = []
            termFreqBlock = {}
            docFreqBlock = {}
                                
    return compressedIndexV1


def commonPrefix(m):
    s1 = min(m)
    s2 = max(m)
    for i, c in enumerate(s1):
        if c != s2[i]:
            return s1[:i]
    return s1

#front encoding compression using gamma coding
def frontCoding_gamma(): #getGammaCode
    k=8
    tempK=0
    termFreqBlock = {}
    deltaEncodingList = []
    docFreqBlock = {}
    termList= []
    tempIndexList = []
    prefix = ""
    temp = ""
    
    for f,term in enumerate(sorted(dictionary_uncomp_v2.keys())):
        if tempK < k:
            termList.append(term)
            tempK += 1
        
        if tempK == k or f == len(dictionary_uncomp_v2)-1 :    
            prefix = commonPrefix(termList)
            if prefix:
                temp += "["
                for n,item in enumerate(termList):
                    if item.startswith(prefix):
                        if n == 0:
                            temp += str(len(item)) + prefix + "*" + item[len(prefix):]
                        if n > 0:
                            temp += str(len(item[len(prefix):])) + "|" + item[len(prefix):]                 
                    else:
                        if n == 0:
                            temp += str(len(item)) + prefix + "*" +  item[:]
                        if n > 0:
                            temp += str(len(item[:])) + "|" + item[:]
                    
                    entry = dictionary_uncomp_v2.get(item)
                    prevId = 0
                    pEntry = PostingEntry(0,0,0,0)
                    
                    for pEntry in entry.postingList:
                        docId = getGammaCode(pEntry.docId - prevId)
                        deltaEncodingList.append(docId)
                        prevId = pEntry.docId
                
                    termFreqBlock[n] = getGammaCode(entry.totTermFreq)
                    docFreqBlock[n] = getGammaCode(entry.docFreq)
               
                temp += "]"
                tempIndexList.append(deltaEncodingList) 
                tempIndexList.append(termFreqBlock)
                tempIndexList.append(docFreqBlock)
                compressedIndexV2[temp] = tempIndexList 
                tempK=0
                temp = ""
                tempIndexList = []
                termList = []
                deltaEncodingList = []
                termFreqBlock = {}
                docFreqBlock = {}
    return compressedIndexV2

#front encoding compression using delta coding
def frontCoding_delta():
    k=8
    tempK=0
    termFreqBlock = {}
    deltaEncodingList = []
    docFreqBlock = {}
    termList= []
    tempIndexList = []
    prefix = ""
    temp = ""
    
    for f,term in enumerate(sorted(dictionary_uncomp_v2.keys())):
        if tempK < k:
            termList.append(term)
            tempK += 1
        
        if tempK == k or f == len(dictionary_uncomp_v2)-1 :    
            prefix = commonPrefix(termList)
            if prefix:
                temp += "["
                for n,item in enumerate(termList):
                    if item.startswith(prefix):
                        if n == 0:
                            temp += str(len(item)) + prefix + "*" + item[len(prefix):]
                        if n > 0:
                            temp += str(len(item[len(prefix):])) + "|" + item[len(prefix):]                 
                    else:
                        if n == 0:
                            temp += str(len(item)) + prefix + "*" +  item[:]
                        if n > 0:
                            temp += str(len(item[:])) + "|" + item[:]
                    
                    entry = dictionary_uncomp_v2.get(item)
                    prevId = 0
                    pEntry = PostingEntry(0,0,0,0)
                    
                    for pEntry in entry.postingList:
                        docId = getGammaCode(pEntry.docId - prevId)
                        deltaEncodingList.append(docId)
                        prevId = pEntry.docId
                
                    termFreqBlock[n] = getDeltaCode(entry.totTermFreq)
                    docFreqBlock[n] = getDeltaCode(entry.docFreq)
               
                temp += "]"
                tempIndexList.append(deltaEncodingList) 
                tempIndexList.append(termFreqBlock)
                tempIndexList.append(docFreqBlock)
                compressedIndexV2[temp] = tempIndexList 
                tempK=0
                temp = ""
                tempIndexList = []
                termList = []
                deltaEncodingList = []
                termFreqBlock = {}
                docFreqBlock = {}
    return compressedIndexV2

#front encoding compression using unary coding
def frontCoding_unary(): #getUnaryValue
    k=8
    tempK=0
    termFreqBlock = {}
    deltaEncodingList = []
    docFreqBlock = {}
    termList= []
    tempIndexList = []
    prefix = ""
    temp = ""
    
    for f,term in enumerate(sorted(dictionary_uncomp_v2.keys())):
        if tempK < k:
            termList.append(term)
            tempK += 1
        
        if tempK == k or f == len(dictionary_uncomp_v2)-1 :    
            prefix = commonPrefix(termList)
            if prefix:
                temp += "["
                for n,item in enumerate(termList):
                    if item.startswith(prefix):
                        if n == 0:
                            temp += str(len(item)) + prefix + "*" + item[len(prefix):]
                        if n > 0:
                            temp += str(len(item[len(prefix):])) + "|" + item[len(prefix):]                 
                    else:
                        if n == 0:
                            temp += str(len(item)) + prefix + "*" +  item[:]
                        if n > 0:
                            temp += str(len(item[:])) + "|" + item[:]
                    
                    entry = dictionary_uncomp_v2.get(item)
                    prevId = 0
                    pEntry = PostingEntry(0,0,0,0)
                    
                    for pEntry in entry.postingList:
                        docId = getUnaryValue(pEntry.docId - prevId)
                        deltaEncodingList.append(docId)
                        prevId = pEntry.docId
                
                    termFreqBlock[n] = getUnaryValue(entry.totTermFreq)
                    docFreqBlock[n] = getUnaryValue(entry.docFreq)
               
                temp += "]"
                tempIndexList.append(deltaEncodingList) 
                tempIndexList.append(termFreqBlock)
                tempIndexList.append(docFreqBlock)
                compressedIndexV2[temp] = tempIndexList 
                tempK=0
                temp = ""
                tempIndexList = []
                termList = []
                deltaEncodingList = []
                termFreqBlock = {}
                docFreqBlock = {}
    return compressedIndexV2

dirPath = "C:\\Users\\Namrata\\Downloads\\cranfield\*"
filesList = glob.glob(dirPath)

#######################################################################################################
compressedIndexV1 = defaultdict()
compressedIndexV2 = defaultdict()
dictionary_uncomp_v1 = dict()
dictionary_uncomp_v2 = dict()
dictWord = defaultdict(int)
stemWord = defaultdict(int)
docList = defaultdict(int)

########################################################################################################
start_time = time.time()
Lemmatiztion()
end_time = time.time()
print("Elapsed time to build Index using lemmatization: version 1: "+str(end_time - start_time)+" seconds") 
start = time.time()
Stemming()
end = time.time()
print("Elapsed time to build Index using lemmatization : version 2: "+str( end - start)+" seconds")
print("")
#########################################################################################################
with open('Index_Version1.uncompress', 'wb') as outfile:
    pickle.dump(dictionary_uncomp_v1, outfile, pickle.HIGHEST_PROTOCOL)
print("Size of the Index version 1 uncompressed (in bytes): " + str(os.path.getsize("Index_Version1.uncompress")))

with open('Index_Version2.uncompress', 'wb') as outfile1:
    pickle.dump(dictionary_uncomp_v2, outfile1, pickle.HIGHEST_PROTOCOL)
print("Size of the Index version 2 uncompressed (in bytes): " + str(os.path.getsize("Index_Version2.uncompress")))
print("")

########################################################################################################
    
blockedCompression_gamma()
with open('Index_Version_blockedCompression_gamma.compress', 'wb') as outfile3:
    pickle.dump(compressedIndexV1, outfile3, pickle.HIGHEST_PROTOCOL)

blockedCompression_delta()
with open('Index_Version_blockedCompression_delta.compress', 'wb') as outfile4:
    pickle.dump(compressedIndexV1, outfile4, pickle.HIGHEST_PROTOCOL)  
    
blockedCompression_unary()
with open('Index_Version_blockedCompression_unary.compress', 'wb') as outfile2:
    pickle.dump(compressedIndexV1, outfile2, pickle.HIGHEST_PROTOCOL) 

##########################################################################################################
frontCoding_gamma()
with open('Index_Version_frontCoding_gamma.compress', 'wb') as outfile5:
   pickle.dump(compressedIndexV2, outfile5, pickle.HIGHEST_PROTOCOL)
    
frontCoding_delta()
with open('Index_Version_frontCoding_delta.compress', 'wb') as outfile5:
   pickle.dump(compressedIndexV2, outfile5, pickle.HIGHEST_PROTOCOL)

frontCoding_unary()
with open('Index_Version_frontCoding_unary.compress', 'wb') as outfile5:
   pickle.dump(compressedIndexV2, outfile5, pickle.HIGHEST_PROTOCOL)

##########################################################################################################

print("Size of the Index compressed using blocked compression & unary coding (in bytes): " + str(os.path.getsize("Index_Version_blockedCompression_unary.compress")))
print("Size of the Index compressed using blocked compression & gamma encoding (in bytes): " + str(os.path.getsize("Index_Version_blockedCompression_gamma.compress")))
print("Size of the Index compressed using blocked compression & delta encoding (in bytes): " + str(os.path.getsize("Index_Version_blockedCompression_delta.compress")))

print("Size of the Index compressed using front coding & unary codes (in bytes): " + str(os.path.getsize("Index_Version_frontCoding_unary.compress")))
print("Size of the Index compressed using front coding & gamma codes (in bytes): " + str(os.path.getsize("Index_Version_frontCoding_gamma.compress")))
print("Size of the Index compressed using front coding & delta codes (in bytes): " + str(os.path.getsize("Index_Version_frontCoding_delta.compress")))


Elapsed time to build Index using lemmatization: version 1: 1.1601014137268066 seconds
Elapsed time to build Index using lemmatization : version 2: 4.350997447967529 seconds

Size of the Index version 1 uncompressed (in bytes): 558283
Size of the Index version 2 uncompressed (in bytes): 417636

Size of the Index compressed using blocked compression & unary coding (in bytes): 380880
Size of the Index compressed using blocked compression & gamma encoding (in bytes): 237252
Size of the Index compressed using blocked compression & delta encoding (in bytes): 239253
Size of the Index compressed using front coding & unary codes (in bytes): 21098
Size of the Index compressed using front coding & gamma codes (in bytes): 11394
Size of the Index compressed using front coding & delta codes (in bytes): 11467
